# Week 4 - Data Processing

이번 시간에는 python의 기본 명령어들을 이용하여 데이터 프로세싱을 하기 위한 기초를 다룬다. 이 과정에서 데이터 파일의 import, 데이터 타입 수정, 분석을 위한 함수 생성, 새로운 컬럼의 생성 등을 다룬다.

먼저 **`table_descriptions.txt`**을 읽어보자.

## Import CSV

unicodecsv라는 모듈은 csv 파일을 unicode로 불러올 수 있게 한다. ```pip install unicodecsv```를 실행하여 해당 모듈을 설치하도록 하자. https://github.com/jdunck/python-unicodecsv

In [1]:
import unicodecsv as csv

enrollments = []  # dictionary를 담을 list
f = open('enrollments.csv', 'rb')
reader = csv.DictReader(f)  # csv를 읽어서 dictionary 형태로 저장한다
for row in reader:
#     print(row)
    enrollments.append(row)
f.close()

print(enrollments[0])

OrderedDict([('account_key', '448'), ('status', 'canceled'), ('join_date', '2014-11-10'), ('cancel_date', '2015-01-14'), ('days_to_cancel', '65'), ('is_udacity', 'True'), ('is_canceled', 'True')])


In [2]:
with open('enrollments.csv', 'rb') as f:
    reader = csv.DictReader(f)
    enrollments = list(reader)
    
print(enrollments[0])

OrderedDict([('account_key', '448'), ('status', 'canceled'), ('join_date', '2014-11-10'), ('cancel_date', '2015-01-14'), ('days_to_cancel', '65'), ('is_udacity', 'True'), ('is_canceled', 'True')])


### 실습 1-1: 다음의 csv 데이터를 불러오자.

daily_engagement.csv 데이터와 project_submissions.csv 데이터를 불러오자.

In [3]:
#================================#
# 실습 1-1                        #
#================================#

# load daily engagement file
with open('daily_engagement.csv', 'rb') as f:
    daily_engagement = list(csv.DictReader(f))
    
# load project submission file
with open('project_submissions.csv', 'rb') as f:
    project_submissions = list(csv.DictReader(f))
    
print(daily_engagement[0])
print(project_submissions[0])

OrderedDict([('acct', '0'), ('utc_date', '2015-01-09'), ('num_courses_visited', '1.0'), ('total_minutes_visited', '11.6793745'), ('lessons_completed', '0.0'), ('projects_completed', '0.0')])
OrderedDict([('creation_date', '2015-01-14'), ('completion_date', '2015-01-16'), ('assigned_rating', 'UNGRADED'), ('account_key', '256'), ('lesson_key', '3176718735'), ('processing_state', 'EVALUATED')])


### 실습 1-2: 위의 코드를 read_csv()라는 함수로 만들어 보자.

In [4]:
#================================#
# 실습 1-2                        #
#================================#

def read_csv(filename):
    with open(filename, 'rb') as f:
        return list(csv.DictReader(f))

enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')

print(enrollments[0])
print(daily_engagement[0])
print(project_submissions[0])

OrderedDict([('account_key', '448'), ('status', 'canceled'), ('join_date', '2014-11-10'), ('cancel_date', '2015-01-14'), ('days_to_cancel', '65'), ('is_udacity', 'True'), ('is_canceled', 'True')])
OrderedDict([('acct', '0'), ('utc_date', '2015-01-09'), ('num_courses_visited', '1.0'), ('total_minutes_visited', '11.6793745'), ('lessons_completed', '0.0'), ('projects_completed', '0.0')])
OrderedDict([('creation_date', '2015-01-14'), ('completion_date', '2015-01-16'), ('assigned_rating', 'UNGRADED'), ('account_key', '256'), ('lesson_key', '3176718735'), ('processing_state', 'EVALUATED')])


## Fixing Data Types

CSV로 불러온 데이터의 value는 모두 string 타입이다. 우리가 데이터를 이용해 뭔가를 계산을 하기 위해서는 데이터의 value를 수정할 필요가 있다. 

예를 들어 enrollment의 데이터 row를 출력하면 다음과 같다.

```
{ 
  'is_udacity': 'True', 
  'cancel_date': '2015-01-14', 
  'days_to_cancel': '65', 
  'account_key': '448', 
  'join_date': '2014-11-10', 
  'is_canceled': 'True', 
  'status': 'canceled'
}
```

이들 데이터 중, cancel_date, join_date은 datetime 데이터 타입으로 저장되어야 날짜와 관련한 연산을 할 수 있다. 또한 days_to_cancel, account_key는 int 타입일 가능성이 높다. is_udacity, is_canceled은 boolean 타입일 것이다. 따라서 이들 데이터를 적절한 데이터 타입으로 바꾸어주어야 한다.

In [5]:
# ====================== #
# 변환을 위한 기본 함수를 작성  #
# ====================== #

from datetime import datetime as dt

# 문자열로 된 날짜를 datetime 형태로 변환한다.
# 변환에 사용된 함수는 strptime()으로 데이터에 있는 날짜 타입에 맞춰 datetime으로 변환한다. 
# (strftime()과 반대의 역할을 하는 함수)
def parse_date(date):
    if date == '':  # data가 없는 경우 None을 저장한다.
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')

# 문자열로 된 정수를 int 타입으로 변환한다.
def parse_int(i):
    if i == '':
        return None
    else:
        return int(i)

# 문자열로 된 실수를 float 타입으로 변환한다.
def parse_float(i):
    if i == '':
        return None
    else:
        return float(i)

# 문자열로 된 bool 값을 bool 타입으로 반환한다.
# "True" -> True, "False" -> False
def parse_bool(b):
    if b == 'True':
        return True
    else:
        return False

In [6]:
# enrollments 데이터 테이블 내의 데이터 타입을 변환
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    enrollment['days_to_cancel'] = parse_int(enrollment['days_to_cancel'])
    enrollment['account_key'] = parse_int(enrollment['account_key'])
    enrollment['is_udacity'] = parse_bool(enrollment['is_udacity'])
    enrollment['is_canceled'] = parse_bool(enrollment['is_canceled'])

print(enrollments[0])

OrderedDict([('account_key', 448), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 1, 14, 0, 0)), ('days_to_cancel', 65), ('is_udacity', True), ('is_canceled', True)])


### 실습 2-1: daily_engagement 데이터의 타입을 고쳐보자.

```
{
  'total_minutes_visited': '11.6793745', 
  'acct': '0', 
  'lessons_completed': '0.0', 
  'projects_completed': '0.0', 
  'num_courses_visited': '1.0', 
  'utc_date': '2015-01-09'
}
```

In [7]:
#================================#
# 실습 2-1                        #
#================================#

for engagement in daily_engagement:
    engagement['total_minutes_visited'] = parse_float(engagement['total_minutes_visited'])
    engagement['acct'] = parse_int(engagement['acct'])
    engagement['lessons_completed'] = parse_int(float(engagement['lessons_completed']))
    engagement['projects_completed'] = parse_int(float(engagement['projects_completed']))
    engagement['num_courses_visited'] = parse_int(float(engagement['num_courses_visited']))
    engagement['utc_date'] = parse_date(engagement['utc_date'])
    
print(daily_engagement[0])

OrderedDict([('acct', 0), ('utc_date', datetime.datetime(2015, 1, 9, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 11.6793745), ('lessons_completed', 0), ('projects_completed', 0)])


### 실습 2-2: project_submissions 데이터의 타입을 고쳐보자.

```
{
  'lesson_key': '3176718735', 
  'creation_date': '2015-01-14', 
  'account_key': '256', 
  'completion_date': '2015-01-16', 
  'processing_state': 'EVALUATED', 
  'assigned_rating': 'UNGRADED'
}
```

In [8]:
#================================#
# 실습 2-2                        #
#================================#

for submission in project_submissions:
    submission['lesson_key'] = parse_int(submission['lesson_key'])
    submission['creation_date'] = parse_date(submission['creation_date'])
    submission['account_key'] = parse_int(submission['account_key'])
    submission['completion_date'] = parse_date(submission['completion_date'])
    
print(project_submissions[0])

OrderedDict([('creation_date', datetime.datetime(2015, 1, 14, 0, 0)), ('completion_date', datetime.datetime(2015, 1, 16, 0, 0)), ('assigned_rating', 'UNGRADED'), ('account_key', 256), ('lesson_key', 3176718735), ('processing_state', 'EVALUATED')])


## Investigating Data

먼저 enrollments, daily_engagement, project_submissions 세개의 데이터에서 unique student number를 찾아보자. (account_key와 set()을 이용)

In [9]:
print(len(enrollments))
print(len(daily_engagement))
print(len(project_submissions))

1640
136240
3642


### 실습 3-1: unique_enrolled_students, unique_engagement_students, unique_project_submitters 의 크기를 구해보자.

In [13]:
#================================#
# 실습 3-1                        #
#================================#

# enrollments
unique_enrolled_students = set()
for enrollment in enrollments:
    unique_enrolled_students.add(enrollment['account_key'])

# daily_engagement
unique_engagement_students = set()
for engagement in daily_engagement:
    unique_engagement_students.add(engagement['acct'])

# project_submissions
unique_project_submitters = set()
for submission in project_submissions:
    unique_project_submitters.add(submission['account_key'])

print(len(enrollments), len(unique_enrolled_students))
print(len(daily_engagement), len(unique_engagement_students))
print(len(project_submissions), len(unique_project_submitters))

1640 1302
136240 1237
3642 743


### 실습 3-2: 똑같은 일을 반복한다면 함수를 만드는 것이 현명하다. unique_students() 함수를 만들어 보자.

In [11]:
#================================#
# 실습 3-2                        #
#================================#

def unique_students(data, key):
    unique_sets = set()
    for row in data:
        unique_sets.add(row[key])
    return len(unique_sets)

print(unique_students(enrollments, 'account_key'))
print(unique_students(daily_engagement, 'acct'))
print(unique_students(project_submissions, 'account_key'))

1302
1237
743


## Problems in the Data

데이터의 unique number를 살펴보니 몇가지 문제점이 보인다.
1. daily_engagement의 account key가 다른 두개의 테이블과 다르다. -> acct를 account_key로 바꾸자.
2. 학생들의 유니크한 숫자가 engagement보다 enrollment에 더 많다. -> engagement는 방문하지 않은 날에도 데이터가 쌓여야 한다. 따라서 erollment와 engagement는 숫자가 같아야 한다. 문제를 찾아보자.

**잘못된 키 수정**

In [14]:
for engagement in daily_engagement:
    engagement['account_key'] = engagement['acct'] # 키를 하나 더 많들고 acct값을 account_key에 assign
    del[engagement['acct']] # acct 키를 삭제

print(daily_engagement[0])

OrderedDict([('utc_date', datetime.datetime(2015, 1, 9, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 11.6793745), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', 0)])


### 실습 4: unique_students() 함수를 수정한 unique_students_list() 함수를 만들어 보자.

키를 수정하고 나서는 위에서 정의한 unique_students()함수를 수정해보자. 이 함수는 위의 함수와 같이 길이를 반환하지 않고 set() 데이터를 반환한다.

In [15]:
#================================#
# 실습 4                          #
#================================#

def unique_students_list(data):
    unique_sets = set()
    for row in data:
        unique_sets.add(row["account_key"])
    return unique_sets

**Missing Engagement Data**

* 이상한 데이터가 있다.
    * enrollment 레코드 중 engagement에 관련 데이터가 없는 것들이 있다.
    * unique account key 를 비교하여 enrollment 에만 있는 key가 있는지 확인하고 데이터를 뽑아보자.

In [16]:
unique_engagement_keys = unique_students_list(daily_engagement)

for enrollment in enrollments:  # enrollment에 있는 요소를 하나씩 꺼낸후
    student = enrollment['account_key']   # account_key의 value를 student 변수에 저장
    if student not in unique_engagement_keys:  # unique_engagement_keys 에 account_key가 없으면
        print(enrollment)  # 해당 요소를 출력한 후 프로그램 종료
        break

OrderedDict([('account_key', 1219), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 12, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 12, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)])


**Findings**
* join_date과 cancel_date이 같은 날이다.
* days_to_cancel이 0이다.
* (참고) days_to_cancel: The number of days between join_date and cancel_date, or blank if the student has not yet canceled.

**해석**
* 일부의 학생은 가입하자마자 cancel을 해서 engagement 테이블에 데이터가 쌓이지 않았다.

**이상한 데이터를 더 찾아보자.**
  * engagement에 없는 enrollment 레코드 중, join_date와 cancel_date가 서로 다른 것들이 있을까?
  * 이 얘기는, 가입하자마자 cancel해서 engagement 테이블에 데이터가 쌓이지 않은 경우가 아닌데도 데이터가 없는 경우이다.

In [17]:
num_of_problem_students = 0

for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_keys and enrollment['join_date'] != enrollment['cancel_date']:
        print(enrollment)
        num_of_problem_students += 1

print("\nNumber of problem students: ", num_of_problem_students)

OrderedDict([('account_key', 1304), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', 1304), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', 1101), ('status', 'current'), ('join_date', datetime.datetime(2015, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_udacity', True), ('is_canceled', False)])

Number of problem students:  3


뭐가 문제일까? 데이터 자체를 봐서는 아무런 문제도 발견되지 않는다.

문제는 is_udacity에 있었다. is_udacity가 True인 경우는 udacity의 테스트 어카운트였다. 따라서 분석을 할 때 is_udacity가 True인 경우 제외하여야 한다.
* (참고) is_udacity: True if the account is a Udacity test account, False otherwise.

**udacity 테스트 어카운트를 제외하자**

먼저 udacity 테스트 어카운트 리스트를 만들자.

In [18]:
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

### 실습 5: udacity test account를 제외하기 위한 함수 remove_udacity_accounts() 를 작성해 보자.

is_udacity가 True인 키를 udacity_test_account에 추가하였다. 모두 6개가 발견되었다. 이들을 제외하기 위한 함수를 작성해보자.

In [19]:
#================================#
# 실습 5                          #
#================================#

def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [20]:
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))

1622
135656
3634


## Data Filtering

**1. 유료사용자 리스트**

udacity에서는 일주일 간의 free trial을 제공하기 때문에 실제로 프로젝트에 참여하고 과정을 마치는 학생들은 유료인 사용자들이다. 맛보기로 과정을 경험해보고자 하는 학생들은 모두 1주일 이내에 과정을 cancel 한다. 그렇다면 유료 사용자의 리스트를 추출해 보자. 유료 사용자의 조건은 다음과 같은 경우일 것이다.

* haven't canceld yet (is_canceled -> False)
* stayed enrolled more than 7 days (days_to_cancel > 7)

데이터를 추출한 후 paid_students라는 딕셔너리를 만들고 다음과 같이 저장하자.
* Keys: account keys 
* Value: enrollment date

### 실습 6: 유료 사용자 paid_students 를 만들고 전체 숫자를 출력해보자.

In [23]:
#================================#
# 실습 6                          #
#================================#

paid_students = {}

for enrollment in non_udacity_enrollments:
    if not enrollment['is_canceled'] or enrollment['days_to_cancel'] > 7:
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        
        # paid_students[account_key] = enrollment_date -> 이렇게 해도 됨. 하지만 아래에서 좀더 경우의 수를 체크.
        
        # account_key 가 paid_students에 이미 들어있지 않거나 enrollment_date가 이미 존재하는 날짜 보다 최근일 경우만 추가
        if account_key not in paid_students or enrollment_date > paid_students[account_key]: 
            paid_students[account_key] = enrollment_date

print(len(paid_students))

995


**2. 첫주 동안 유료 사용자의 engagement데이터 리스트**

다음의 함수는 학생들의 enrollment join date와 특정한 engagement 레코드의 날짜를 이용해서 join 후 일주일 이내의 것인지를 판별하여주는 코드이다. 다음을 이용하여 유료 사용자의 첫주간의 engagement 데이터 리스트를 추출해보자.

In [24]:
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

먼저 fee trial 데이터를 제외하는 함수를 작성하자.

위에서 `paid_student` 를 구해두었기 때문에 이를 활용하면 된다.

In [25]:
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

다음은 위의 함수를 이용하여 유료사용자의 데이터인 paid_enrollments, paid_engagement, paid_submissions을 구해보자

In [26]:
paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)

print(len(paid_enrollments))
print(len(paid_engagement))
print(len(paid_submissions))

1293
134549
3618


마지막으로 `within_one_week()`함수를 사용하여 일주일 동안의 데이터를 추려내 보자.

### 실습 7: paid_engagement_in_first_week 를 구하는 코드를 작성해보자.

* paid_engagement_in_first_week는 리스트로 생성한다.
* paid_engagement에서 'accont_key'를 가져와서 이를 이용하여 paid_students의 join_date을 찾아냔다.
* paid_engagement의 'utc_date'을 이용하여 engagement_date을 찾아낸다.
* 마지막으로 within_one_week() 함수를 사용하여 engagement가 일주일 이내에 작성된 것들만 추려 paid_engagement_in_first_week 에 저장한다.

In [27]:
#================================#
# 실습 7                          #
#================================#

paid_engagement_in_first_week = []

for engagement in paid_engagement:
    account_key = engagement['account_key']  # 각 engagement데이터의 account_key 값을 저장한다.
    join_date = paid_students[account_key]   # 저장된 account_key 값을 이용해 사용자의 join_date를 찾는다. paid_students는 id-join_date 형태로 위에서 만들어 두었다.
    engagement_date = engagement['utc_date'] # engagement데이터의 date을 engagement_date에 저장한다.
    
    if within_one_week(join_date, engagement_date):
        paid_engagement_in_first_week.append(engagement)

print(len(paid_engagement_in_first_week))

21508


## Exploring Student Engagement

defaultdict()는 dictionary에 기본값을 정의하고 키값이 없더라도 에러를 출력하지않고 기본값을 출력한다. 따라서 빈도수 등을 계산할 때 키값이 없어도 키를 만들 수 있다.

In [28]:
from collections import defaultdict

document = ["a", "b", "c", "a", "a", "c", "a", "c", "a", "a", "a", "b", "c", "b"]
word_counts = defaultdict(int) # value를 int로 지정
for word in document:
    word_counts[word] +=1
print(word_counts)

defaultdict(<class 'int'>, {'a': 7, 'b': 3, 'c': 4})


In [29]:
dd_list = defaultdict(list)
dd_list["key1"].append([1,2])
dd_list["key1"].append([2,3])
dd_list["key2"].append(1)
print(dd_list)

defaultdict(<class 'list'>, {'key1': [[1, 2], [2, 3]], 'key2': [1]})


account_key를 key로 하고 engagement를 value로 하는 dictionary list를 만들어 보자. account_key를 기준으로 얼마나 많은 engagement가 기록되었는지를 확인할 수 있다. key가 없으면 empty dictionary를 만든다.

In [36]:
from collections import defaultdict

engagement_by_account = defaultdict(list)
for engagement in paid_engagement_in_first_week:
    account_key = engagement['account_key']
    engagement_by_account[account_key].append(engagement)
    
print(engagement_by_account[0])

[OrderedDict([('utc_date', datetime.datetime(2015, 1, 9, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 11.6793745), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', 0)]), OrderedDict([('utc_date', datetime.datetime(2015, 1, 10, 0, 0)), ('num_courses_visited', 2), ('total_minutes_visited', 37.2848873333), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', 0)]), OrderedDict([('utc_date', datetime.datetime(2015, 1, 11, 0, 0)), ('num_courses_visited', 2), ('total_minutes_visited', 53.6337463333), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', 0)]), OrderedDict([('utc_date', datetime.datetime(2015, 1, 12, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 33.4892696667), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', 0)]), OrderedDict([('utc_date', datetime.datetime(2015, 1, 13, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 64.7796776667), ('lessons_completed', 0), ('

In [37]:
total_minutes_by_account = {}

for account_key, engagement_for_students in engagement_by_account.items(): # dictvalue의 각 item을 이용하여 iteration을 하기 위해 items() 사용
    total_minutes = 0
    for engagement in engagement_for_students:
        total_minutes += engagement['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes
    
print(total_minutes_by_account)

{0: 494.88049616599994, 1: 18.576384666670002, 2: 0.0, 3: 2530.6469816678004, 4: 33.3214046667, 5: 329.7578566663, 6: 780.4545511666701, 7: 104.20388850009999, 8: 989.113641833, 9: 448.471384167, 10: 130.12347833367, 11: 0.0, 12: 179.4719088333, 13: 1013.3833969996999, 14: 65.6221875, 15: 1042.4943771683002, 16: 119.12030049999998, 17: 235.49969150033, 18: 155.1361575, 19: 0.0, 20: 447.93897783336996, 21: 931.1036911666699, 22: 657.2052335000001, 23: 1591.3228143334, 24: 943.188117167, 25: 2997.70812683204, 26: 430.801675833, 27: 1579.12122666663, 28: 766.256315667, 29: 556.1906033333, 30: 69.6578351667, 31: 1917.2210374995, 32: 123.2915048333, 33: 253.9870258334, 34: 478.4825176664, 35: 765.6402170004, 36: 809.2138958339, 37: 1378.195091668, 38: 0.0, 39: 1001.5888595, 40: 478.21952616690004, 41: 511.925391, 42: 1898.9588304992003, 43: 88.822038, 44: 2783.6493991655298, 45: 3933.373638167089, 46: 26.1189351667, 47: 854.8288881656, 48: 1055.6871896667, 49: 0.0, 50: 491.9087078335, 51: 2

**numpy의 사용**

다음 시간에 자세히 다루겠지만, numpy는 mean, sd 등의 계산을 쉽게 해준다. numpy를 이용하여 평균을 구해보자.

In [38]:
import numpy as np

test_list = [1,2,3,4,5,6,7,8,9,10]
print(np.mean(test_list))
print(np.sum(test_list))
print(np.std(test_list))
print(np.max(test_list), np.min(test_list))

5.5
55
2.8722813232690143
10 1


In [41]:
import numpy as np

# 모든 어카운트의 전체 사용시간 (value로 저장된 내용)을 value를 이용하여 리스트로 만든다.
total_minutes = total_minutes_by_account.values()

# print(np.mean(total_minutes)) # 에러가 남. total_minutes은 list가 아니라 dict value임. 따라서 list로 변환해 주어야 함.

print('Mean:', np.mean(list(total_minutes)))
print('Standard deviation:', np.std(list(total_minutes)))
print('Minimum:', np.min(list(total_minutes)))
print('Maximum:', np.max(list(total_minutes)))

Mean: 647.5901738262695
Standard deviation: 1129.2712104188108
Minimum: 0.0
Maximum: 10568.100867332541


계산 결과 다음을 확인할 수 있었다. 
* 학생들이 강의에 engage한 시간은 평균 647.59분이었다.
* 편차는 1229.27분으로 매우 큰 편이다. 
* 가장 적게 이용한 학생이 0분, 가장 많이 이용한 학생은 10568분이다.

그런데, 이 결과는 다시 의문을 만들게 된다. 10568분은 1주일이 넘는 시간이다. 이 데이터는 학생들의 첫번째 주 engagement를 계산한 것인데, 어느 학생도 일주일 내내 24시간 강의서비스를 이용하지도 않았을 것이고, 심지어 1주일이 넘는 시간이 발견되었다. 문제는 무엇일까?

## Debugging the Codes

문제가 발생한 곳을 찾기 위해서 max_minutes이 10568.10인 학생의 key를 찾아보자.

In [42]:
student_key_with_max_minutes = None
max_minutes = 0

for student_key, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes
        student_key_with_max_minutes = student_key

print(max_minutes, student_key_with_max_minutes)

10568.100867332541 108


위에서 확인한 키를 이용하여 engagement 데이터를 출력해보자.

In [43]:
for engagement in paid_engagement_in_first_week:
    if engagement['account_key'] == student_key_with_max_minutes:
        print(engagement)

OrderedDict([('utc_date', datetime.datetime(2015, 1, 7, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 50.9938951667), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', 108)])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 8, 0, 0)), ('num_courses_visited', 2), ('total_minutes_visited', 688.3034385), ('lessons_completed', 5), ('projects_completed', 0), ('account_key', 108)])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 9, 0, 0)), ('num_courses_visited', 2), ('total_minutes_visited', 427.691757667), ('lessons_completed', 1), ('projects_completed', 0), ('account_key', 108)])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 10, 0, 0)), ('num_courses_visited', 3), ('total_minutes_visited', 165.6270925), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', 108)])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 11, 0, 0)), ('num_courses_visited', 0), ('total_minutes_visited', 0.0), ('lessons_completed', 0), ('projects_c

너무 많은 engagement data가 출력이 되었다. 우리가 일주일치의 유료사용자 데이터만을 필터링했기 때문에 데이터는 7개가 있어야 한다. 심지어 날짜를 보면 1월부터 4월까지의 데이터가 수집되었다. 아마도 within_one_week() 함수에 문제가 있는 것 같다.

In [44]:
for enrollment in paid_enrollments:
    if enrollment['account_key'] == student_key_with_max_minutes:
        print(enrollment)

OrderedDict([('account_key', 108), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 7, 0, 0)), ('cancel_date', datetime.datetime(2015, 4, 20, 0, 0)), ('days_to_cancel', 103), ('is_udacity', False), ('is_canceled', True)])
OrderedDict([('account_key', 108), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 4, 20, 0, 0)), ('cancel_date', datetime.datetime(2015, 5, 14, 0, 0)), ('days_to_cancel', 24), ('is_udacity', False), ('is_canceled', True)])


In [45]:
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

위의 코드에서는 time_delta.days가 7 이하인 모든 것을 수집한다. 하지만 가입을 했다 탈퇴하고 다시 가입하는 경우 join_date 이전의 기록이 있을 수 있다. 따라서 engagement 데이터는 join date와 같은 날이거나 7일 이내이어야 한다. 따라서 time_delta 는 다음과 같이 수정해야 한다.

0 <= time_delta.days < 7

In [46]:
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return 0 <= time_delta.days < 7

## Lesson completed in first week

### 실습 8: 다음의 코드를 수정하여 lessons completed in first week 를 계산해보자.

In [49]:
#================================#
# 실습 8-code1                    #
#================================#

from collections import defaultdict

engagement_by_account = defaultdict(list)
for engagement in paid_engagement_in_first_week:
    account_key = engagement['account_key']
    engagement_by_account[account_key].append(engagement)

In [52]:
#================================#
# 실습 8-code2                    #
#================================#

total_minutes_by_account = {}

for account_key, engagement_for_students in engagement_by_account.items():
    total_minutes = 0
    for engagement in engagement_for_students:
        total_minutes += engagement['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes
total_lessons_by_account = {}

for account_key, engagement_for_students in engagement_by_account.items(): 
    total_lessons = 0
    for engagement in engagement_for_students:
        total_lessons += engagement['lessons_completed']
    total_lessons_by_account[account_key] = total_lessons

In [54]:
#================================#
# 실습 8-code3                    #
#================================#

import numpy as np

total_minutes = total_minutes_by_account.values()
print('Mean:', np.mean(list(total_minutes)))
print('Standard deviation:', np.std(list(total_minutes)))
print('Minimum:', np.min(list(total_minutes)))
print('Maximum:', np.max(list(total_minutes)))

import numpy as np

total_lessons = total_lessons_by_account.values()
print('Mean:', np.mean(list(total_lessons)))
print('Standard deviation:', np.std(list(total_lessons)))
print('Minimum:', np.min(list(total_lessons)))
print('Maximum:', np.max(list(total_lessons)))

Mean: 647.5901738262695
Standard deviation: 1129.2712104188108
Minimum: 0.0
Maximum: 10568.100867332541
Mean: 3.0924623115577887
Standard deviation: 5.362768866632357
Minimum: 0
Maximum: 43


### 실습 9: 위의 코드들은 이미 두번이나 비슷하게 사용했다. 아마도 계속 사용하게 된다면 함수를 만들어 두는 것이 편리할 것이다. 위의 과정을 수행하는 함수를 작성해 보자.

In [55]:
#================================#
# 실습 9                          #
#================================#

from collections import defaultdict
import numpy as np

def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key].append(data_point)
    return grouped_data

def sum_grouped_items(data, key_name):
    summed_data = {}
    
    for key, data_points in data.items(): 
        total = 0
        for data in data_points:
            total += data[key_name]
        summed_data[key] = total
    return summed_data

def describe_data(data):
    print('Mean:', np.mean(list(data)))
    print('Standard deviation:', np.std(list(data)))
    print('Minimum:', np.min(list(data)))
    print('Maximum:', np.max(list(data)))

In [56]:
engagement_by_account = group_data(paid_engagement_in_first_week, 'account_key')
total_minutes_by_account = sum_grouped_items(engagement_by_account, 'total_minutes_visited')
total_minutes = total_minutes_by_account.values()
describe_data(total_minutes)

total_lessons_by_account = sum_grouped_items(engagement_by_account, 'lessons_completed')
total_lessons = total_lessons_by_account.values()
describe_data(total_lessons)

Mean: 647.5901738262695
Standard deviation: 1129.2712104188108
Minimum: 0.0
Maximum: 10568.100867332541
Mean: 3.0924623115577887
Standard deviation: 5.362768866632357
Minimum: 0
Maximum: 43
